# Asyncio - Coroutine, Future, Task, Event Loop

https://pymotw.com/3/asyncio/index.html  
http://kunkle.org/blog/2012/04/13/nodejs-basics-explained/  

https://medium.com/@yeraydiazdiaz/asyncio-for-the-working-python-developer-5c468e6e2e8e#.ai2lhtm6n  

http://masnun.com/2015/11/20/python-asyncio-future-task-and-the-event-loop.html  
http://quietlyamused.org/blog/2015/10/02/async-python/  

In [3]:
import asyncio

## Introduction

### Asynchronous I/O

Async is shorthand for *cooperative multitasking*, a style of multitasking where the currently running task does not get interrupted, but must yield control to other tasks (`Coroutine`). In the normal case, it may fire off request and then sleep until the request comes back with data (`Future`). During the time that it sleeps, other tasks may run and submit their requests in the same way.

### Concurrency vs Parallelism

https://www.youtube.com/watch?v=cN_DpYBzKso
http://www.slideshare.net/dabeaz/an-introduction-to-python-concurrency

- *Concurrency*: coordination of independently executed things (ie async functions)
    - Goal = dealing with a lot of things at once
    - concurrent: opposite of sequencial (following a logical order/sequence)
- *Parallelism*: simultaneous execution of multiple things (related or not)
    - Goal = doing a lot of things at once
    - parallel: opposite of serial (taking place in a series)

Concurrency is about structuring things so that maybe you can do parallelism.

Breaking down tasks into subtasks only *allows* parallelism, its

```
# sequential
[1....][2........]

# parallel
[1....]
[2........]

# concurrent
[1..[[2........]]1..]
```

### asyncio

Asynchronous programming in Python revolves around four major components:

- A *coroutine* is an asynchronous function. They can be paused and resumed later in time. When paused, they release control back to the caller (aka event loop) without losing their state.
- A *future* represents a result that may or may not come later in time.


- A *task* is a future which wraps and manages a coroutine.
- An *event loop* manages tasks (registering, scheduling, delaying, executing). It handles distributing the flow of control between tasks: context switch.

## Coroutine (asynchronous function)

An **asynchronous function** in Python is typically called a *coroutine*.

Put simply, a coroutine is a function which **can be paused and then resumed later**.  
While a coroutine is waiting, control is returned to the event loop so that other things can run !

With asynchronous programming, you allow your code to handle other tasks while waiting for some tasks to respond.

The **input** to a coroutine is **desynchronized** from the **output** returned (= asynchronous function = asynchronous input/output = asyncio).
- *Asynchronous*: not related in time.

NB: coroutines can only be called within other coroutines or by the event loop as tasks.

### async

Use the `async` keyword to define an asynchronous function (coroutine).

In [ ]:
async def task_double(x):
    return x * 2

### await

Use the `await` keyword to make a **call to another asynchronous function** and **wait for its output**.

By using `await`, the coroutine lets the event loop know that it may switch context to the next task scheduled for execution (if any).

In [ ]:
async def task_double(x):
    await asyncio.sleep(1)  # pause for 1 second, control is 
    return x * 2

## Future

A `Future` object is like a **place holder for a result that should come in the future** (like a `Promise` object in Javascript). A future represents the result of a work that has not been completed yet.

A future can be in one of four states:
- pending
- running
- done
- cancelled

### Future API

In [ ]:
# instanciation
future = asyncio.Future()
future = loop.create_future()  # prefered way for optimization (cf. Event Loop)

# future state
future.done()  # True if the future is done
future.cancelled() # True if the future was cancelled

# future manipulation on success (promise resolved)
future.set_result(result)  # success: mark as done, and set its result
future.result()  # return the result of this future

# future manipulation on fail (promise rejected)
future.set_exception(exception) # fail: mark as done, and set an exception
future.cancel()  # cancel the future and schedule callbacks
future.exception() # return the exception that was thrown by the future

# future callbacks (functions called when future is done, aka promise resolved)
future.add_done_callback(func)  # add a callback to run when the future is done
future.remove_done_callback()  # removes all callback functions

## Task = Future + Coroutine

A `Task` is a `Future` that wraps a `Coroutine`.  
When the coroutine finishes, the result of the task is realized.

## Event Loop

The event loop is responsible for **scheduling and executing tasks** (aka registering, delaying, executing and cancelling tasks). Generally, we schedule multiple tasks to the event loop.  

Event loops use *context switch* for *cooperative scheduling* :
- An event loop **only runs one task at a time**.
- **While a task waits** for the completion of a future, the event loop **executes another task**.  

The event loop provides other features:
- Delegate function calls to a pool of threads
- Create client and server transports for communication
- Create subprocesses and transports for communication with another program  

NB: Other tasks may run in parallel if other event loops are running in different threads.

![event-loop](http://misclassblog.com/wp-content/uploads/2013/04/event-loop.jpg)


### Event Loop API

In [ ]:
event_loop = asyncio.get_event_loop()

# event loop status
event_loop.is_running()  # True if the event loop is running
event_loop.is_closed()   # True if the event loop is closed

# running an event loop
event_loop.run_until_complete(future)  # run until future is done
event_loop.run_forever()  # run until .stop() is called

# stopping an event loop
event_loop.stop()  # stop executor asap
event_loop.close()  # clear the task queue + shut down the executor immediately (pending callbacks are lost)

# futures and tasks
event_loop.create_future()
event_loop.create_task(coroutine)  # make task from coroutine, and schedule it

# event loop calls
event_loop.call_soon(callback)  # schedules a callback to be called asap
event_loop.call_at(when, callback)  # schedules a callback to be called at a given timestamp
event_loop.call_later(delay, callback)  # schedules a callback to be called after a given timeout
event_loop.time()  # return the current time of the event loop's internal clock

### Examples

#### Define coroutines

In [4]:
async def foo():
    print('Running foo')
    await asyncio.sleep(.5)
    print('Running foo again')
    return 1


async def bar():
    print('Running bar')
    await asyncio.sleep(1)
    print('Running bar again')
    return 2

#### Gathering results from coroutines

In [ ]:
event_loop = asyncio.get_event_loop()

# gather all tasks results in a future
future = asyncio.gather(foo(), bar())

# complete tasks
results = event_loop.run_until_complete(future)
print(results)

#### Waiting for multiple coroutines

In [ ]:
event_loop = asyncio.get_event_loop()

# make tasks out of coroutines
coroutines = [foo(), bar()]
tasks = [event_loop.create_task(coroutine) for coroutine in coroutines]
future = asyncio.wait(tasks, return_when='ALL_COMPLETED')  # try FIRST_COMPLETED, ALL_COMPLETED, FIRST_EXCEPTION

# complete tasks
done, pending = event_loop.run_until_complete(future)
print('TASKS DONE : ', done)
print('TASKS PENDING : ', pending)


#### Feed tasks to a forever event loop

In [10]:
event_loop = asyncio.get_event_loop()

# make tasks out of coroutines
coroutines = [foo(), bar()]
tasks = [event_loop.create_task(coroutine) for coroutine in coroutines]

# complete tasks
event_loop.run_forever()
event_loop.stop()

Running foo
Running bar
Running foo again
Running bar again


KeyboardInterrupt: 

#### Future callbacks

In [7]:
def print_result(future):
    print('RESULT OF TASK: ', future.result())

loop = asyncio.get_event_loop()

# create a task from a coroutine object
task = loop.create_task(foo())

# set callback when task is done
task.add_done_callback(print_result)

loop.run_until_complete(task)

Running foo
Running foo again
RESULT OF TASK:  1


1